<a href="https://colab.research.google.com/github/cclljj/LJ-test/blob/master/6_2_%E7%95%B0%E5%B8%B8%E5%81%B5%E6%B8%AC_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Url: 
- [https://github.com/cclljj/TW-Civil-IoT-2020/tree/master/2022](https://github.com/cclljj/TW-Civil-IoT-2020/tree/master/2022)
- [https://github.com/cclljj/TW-Civil-IoT-2028/tree/master/2022](https://github.com/cclljj/TW-Civil-IoT-2018/tree/master/2022)

Location area: `Lat: [22.631231, 22.584989]`, `Lon: [120.263422, 120.346764]`

Date range: 2022.10.15 - 2022.10.28

# 6.2 - 異常偵測

**作者**：羅泉恆 <br>
**更新時間**: 2022.11.13


---

目前已有多個大型低成本空品監測系統成功部署於不同國家與城市之中，然而低成本感測器的主要挑戰之一為確保數據質量。台灣中央研究院資訊科學研究所網路實驗室提出了一種可用於實際環境中的異常檢測框架，稱之為 [ADF, Anomaly Detection Framework](https://ieeexplore.ieee.org/document/8081731)。

此框架由四個模塊所組成：

1.   **TSAD, Time-Sliced Anomaly Detection 時間片斷異常偵測**<br>
     即時偵測感測器於空間或時間上的異常數據，例如是否為室內機器、故障機器
    

2.   **Real-time Emission Detection 即時污染偵測**<br>
     檢測潛在的區域污染源
    

3.   **Device Ranking 感測器可靠度評估**<br>
     為每個微型感測器評估其設備的可靠度


4.   **Malfunction Detection 故障機器偵測**<br>
     識別故障的微型感測器



### 如何識別異常雜訊

* **空間**<br>
我們可以假設戶外的空氣在地理空間上是會均勻擴散的，也就是附近的空氣品質應該要是「ㄧ致」的，不該在短距離出現劇烈變化，例如某台空氣盒子偵測到了高濃度的 PM2.5 數值，但同一時間其方圓 3 公里範圍的其他五台機器偵測的濃度很低，就可機會為異常狀況。

* **時間**<br>
再來我們亦假設空氣的擴散是緩慢的，同一台的空氣盒子在短時間內的數值應為「一致」，如果有台空氣盒子的 PM2.5 濃度突然地劇烈變化，也可能代表著有異常狀況發生。

### 常見因素

而會發生異常的情況有許多種原因，常見的因素有

1. 安裝環境：安裝於廟旁、燒烤店、室內不通風環境

2. 機器故障：感測器取風口裝反、風扇卡了過多灰塵

3. 臨時污染源：旁邊有人在抽菸、發生火災、（非法）排放污染物

---

在這篇文章中，我們將以民生公共物聯網中的空品資料為例，取部分佈建於高雄的微型空品感測器（AirBox）來進行分析，介紹如何使用 ADF 檢測框架來找出其中可能為室內機器或污染源附近的機器，藉此過濾出可信度相對較低的機器們，進而提高整體空品感測結果的可信度。

我們可以透過比較每台空氣盒子與鄰近機器以及相近時間內的一致性，來進行異常偵測的判斷。ADF 每 24 小時會執行一次評分，分析過去 14 天的資料，來推斷異常狀況（indoor, emission）。


核心觀念為，將微型感測器本身的 PM2.5 數值與其周圍 3 公里內的其他感測器進行比較，如果本身數值**低於**周圍平均一定程度標記為`indoor`，高於則標記為`emission`。在開始運算之前，有幾個觀念需要先建立,

### SPATIAL_THRESHOLD

對於不同濃度的 PM2.5 數值，高於和低於的標準應有所不同，越低的數值應有越小的範圍、越高的數值應有越大的範圍。

在以下實驗中，我們將 threshold 定義如下：

| 原始數值 (ppm) | Threshold |
|---------|-----------|
| 0-11 | 6.6 |
| 12-23 | 6.6 |
| 24-35 | 9.35 |
| 36-41 | 13.5 |
| 42-47 | 17.0 |
| 48-58 | 23.0 |
| 59-64 | 27.5 |
| 65-70 | 33.5 |
| 71+ | 91.5 |

舉例來說，當原始數值為 10 (ppm) 時，周圍感測器平均需高於 `10+6.6` ，我們才會將該時間段加入後續標記的參考。

---
## 套件安裝與引用

在本章節中，我們將會使用到 pandas, numpy, plotly 和 geopy 等套件，這些套件在我們使用的開發平台 Colab 上已有預先提供，因此我們不需要另行安裝，可以直接用下列的方法引用，以備之後資料處理與分析使用。

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px # 繪製地圖
from geopy.distance import geodesic # 計算距離

## 讀取資料與環境設定

在以下範例中，我們將取部分民生公共物聯網佈建於高雄的微型空品感測器（AirBox）來進行分析，

- 地理區域：緯度: `22.631231 - 22.584989`, 經度: `120.263422 - 120.346764`
- 時間區間：2022.10.15 - 2022.10.28

PS. AirBox 原始資料請至 [民生公共物聯網-資料服務平台](https://ci.taiwan.gov.tw/dsp/)下載，後續步驟所使用的 `allLoc.csv` 檔案為預先下載並過濾的結果。


In [ ]:
## 執行這個儲存格以掛接 Google 雲端硬碟。
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
## 載入資料
DF = pd.read_csv("allLoc.csv")
DF.head()

,device_id,date,time,temperature,RH,PM2.5,lat,lon
0,74DA38F20F0C,2022-10-15,00:00:05,27.12,100.0,0.0,22.624,120.271
1,74DA38F207DE,2022-10-15,00:00:21,26.12,100.0,38.0,22.603,120.340
2,74DA38F20D8A,2022-10-15,00:01:02,27.00,100.0,32.0,22.606,120.337
3,74DA38F20A10,2022-10-15,00:01:45,25.75,100.0,19.0,22.631,120.311
4,74DA38F20B20,2022-10-15,00:02:14,25.25,98.0,0.0,22.626,120.304


In [ ]:
## 計算感測器 GPS 資訊，將個感測器此時間區間的經緯度欄位取平均
dfId = DF[["device_id","lon","lat"]].groupby("device_id").mean().reset_index()
dfId.head()

,device_id,lon,lat
0,74DA38F207DE,120.340,22.603
1,74DA38F20A10,120.311,22.631
2,74DA38F20B20,120.304,22.626
3,74DA38F20B80,120.350,22.599
4,74DA38F20BB6,120.324,22.600


In [ ]:
## (番外步驟) 將感測器們顯示於地圖上
fig_map = px.scatter_mapbox(dfId, lat="lat", lon="lon",
                  color_continuous_scale=px.colors.cyclical.IceFire, zoom=9,
                  mapbox_style="carto-positron")
fig_map.show()

## 尋找鄰居 (鄰近的感測器)

接著，因為是透過民生公共物聯網計畫所佈建的感測器，感測器多位於校園內，GPS 位置不會有大幅度的變化。為簡化後續的步驟，我們先統一將 “鄰居” 列表計算出來。

**鄰居的定義**：直線距離不超過 3 公里

`DISTENCE = 3`

In [ ]:
## 我們撰寫一個 Python 的小函式 countDis，可以針對輸入的兩條 GPS 資訊，計算兩者間的直線距離，並回傳以公里為單位的結果。

def countDis(deviceA, deviceB):
    return geodesic((deviceA["lat"], deviceB['lon']), (deviceB["lat"], deviceB['lon'])).km

In [ ]:
## 設定範圍為 3 公里內
DISTENCE = 3

## 轉化 Dataframe -> Dictionary
# {iD: {'lon': 0.0, 'lat': 0.0}, ...}
dictId = dfId.set_index("device_id").to_dict("index")

## 建立感測器列表
listId = dfId["device_id"].to_list()

## 初始化鄰居字典（dictionary）
# {iD: []}
dicNeighbor = {iD: [] for iD in listId}

## 依序帶入 countDis Function 計算距離
# 被避免重複的計算，當兩者距離小於 3 時，將彼此加入彼此的鄰居名單中
# Time complexity: N!
for x in range(len(listId)):
    for y in range(x+1, len(listId)):
        if ( countDis( dictId[listId[x]], dictId[listId[y]]) < DISTENCE ):
            dicNeighbor[listId[x]].append( listId[y] )
            dicNeighbor[listId[y]].append( listId[x] )

---
## 計算 5 分鐘平均值
為了讓感測器與鄰居的比較更佳準確，我們將時間區段以五分鐘為單位進行切割，計算各感測器各時間段的平均。

時間段設定: `FREQ = '5min'`

In [ ]:
## 製作 datetime 型態的時間欄位

# 將 date 與 time 欄位組合成 datetime 欄位
DF["datetime"] = DF["date"] + " " + DF["time"]

# 捨棄不需要的 date, time columns
# inplace=True 表示將計算過後的資料取代原有的
DF.drop(columns=["date","time"], inplace=True)

# 轉換 datetime 欄位的型態為時間（datetime）型態
DF['datetime'] = pd.to_datetime(DF.datetime)

DF.head()

,device_id,temperature,RH,PM2.5,lat,lon,datetime
0,74DA38F20F0C,27.12,100.0,0.0,22.624,120.271,2022-10-15 00:00:05
1,74DA38F207DE,26.12,100.0,38.0,22.603,120.340,2022-10-15 00:00:21
2,74DA38F20D8A,27.00,100.0,32.0,22.606,120.337,2022-10-15 00:01:02
3,74DA38F20A10,25.75,100.0,19.0,22.631,120.311,2022-10-15 00:01:45
4,74DA38F20B20,25.25,98.0,0.0,22.626,120.304,2022-10-15 00:02:14


In [ ]:
## 計算平均

# 捨棄不需要的 RH, temperature, lat, lon columns
# - RH, temperature： 本次只需要 PM2.5 的數值
# - lat, lon： GPS 位置不需要，已經在上方步驟計算完鄰居列表了
DF.drop(columns=["RH","temperature","lat","lon"], inplace=True)

# Groupy, 以感測器 id 和五分鐘時間單位進行分組，以此計算每一組的平均
# 可以自行調整適合的時間區間長度
FREQ = '5min'
dfMean = DF.groupby(['device_id', pd.Grouper(key = 'datetime', freq = FREQ)]).agg('mean')

# 去除 PM2.5 欄位為負的 rows (<0)
dfMean = dfMean[ dfMean['PM2.5'] >= 0 ]
dfMean

# 值得注意的部分：
# dataframe 的 index 已經不是單純的一欄 (0,1,2,3 ...)，而是同時以 device_id 和 datetime 作為 index，我們稱之為 MultiIndex

PM2.5
device_id    datetime                  
74DA38F207DE 2022-10-15 00:00:00   38.0
             2022-10-15 00:05:00   37.0
             2022-10-15 00:10:00   37.0
             2022-10-15 00:20:00   36.0
             2022-10-15 00:25:00   36.0
...                                 ...
74DA38F210FE 2022-10-28 22:55:00    3.0
             2022-10-28 23:00:00    3.0
             2022-10-28 23:10:00    3.0
             2022-10-28 23:20:00    2.0
             2022-10-28 23:45:00    2.0

[52523 rows x 1 columns]

---
## 計算鄰居感測器的 PM2.5 平均值


In [ ]:
# 建立兩個 function 來處理鄰居 PM2.5 數值的計算

def cal_mean(iD, dt):
  
  # 取出符合條件的 rows 來計算 "PM2.5" 欄位的平均
  # 條件 1: 'datetime' 相同
  # 條件 2: 'device_id' 在 鄰居列表中 
  # index.get_level_values(0) 與 index.get_level_values('device_id') 有相似的功能，可以想像都是取用 index 的值
  neighborPM25 = dfMean[ 
              (dfMean.index.get_level_values('datetime') == dt) 
                  & (dfMean.index.get_level_values(0).isin(dicNeighbor[iD])) 
          ]["PM2.5"]

  # 取平均
  avg = neighborPM25.mean()

  # 計算該時段鄰居數量
  neighbor_num = neighborPM25.count()

  # 回傳鄰居 PM2.5 平均數值與鄰居數量
  return avg, neighbor_num

def apply_func(x):
  return cal_mean(x.name[0], x.name[1])

In [ ]:
# 這邊使用了 zip, apply 語法來將 dataframe 的數值帶入 function 中
# 使用 zip 是因為 apply_func 會回傳兩項參數，分別是對應 'avg' 和 'neighbor_num' 兩個欄位，在 Pyhton 中需要使用 zip(*Function) 語法來對應回傳和 assign
# 而 apply 則是為了配合 pandas dataframe 的處理

dfMean['avg'], dfMean['neighbor_num'] = zip(*dfMean.apply(apply_func, axis=1))
dfMean

PM2.5        avg  neighbor_num
device_id    datetime                                           
74DA38F207DE 2022-10-15 00:00:00   38.0  13.400000            10
             2022-10-15 00:05:00   37.0  19.888889             9
             2022-10-15 00:10:00   37.0  16.500000            12
             2022-10-15 00:20:00   36.0  16.750000             8
             2022-10-15 00:25:00   36.0  17.000000            11
...                                 ...        ...           ...
74DA38F210FE 2022-10-28 22:55:00    3.0  16.666667            12
             2022-10-28 23:00:00    3.0  19.642857            14
             2022-10-28 23:10:00    3.0  18.307692            13
             2022-10-28 23:20:00    2.0  19.545455            11
             2022-10-28 23:45:00    2.0  16.615385            13

[52523 rows x 3 columns]

---
## 資料分析
如前言所述，不能單以特定的時間或高於低於就判斷其為室內或是鄰近污染源感測器

首先，引入 threshold 的概念，在各小區段進行標記

接著，將時間區間再次延伸至 1 day, 7 days, 14 days，舉例來說，一天內會有 24 小時，如果資料都沒有缺失將會有 288 個小單位，ADF 演算法會以 1/3 為基準，亦即當有 288*(1/3) = 96 個單位被標記為異常，得以進入下一階段的分析，如果小於該基準則不會被帶入下一階段。

最後一步驟，便是將上個步驟中的三段時間區間整合，ADF 中設定各自比例為

|Period (days)|Weights||
|---|---|---|
|1|0.2|A|
|7|0.3|B|
|14|0.5|(1-A-B)|

可以理解為 `0.2 * 一天的可能性` + `0.3 * 七天的可能性` + `0.5 * 十四天的可能性` = 最終可能性，而最終可能性如果高於**某一程度**，我們才會將其標上可能為異常的感測器。

上文中某一程度，可以很直覺地用以下方式計算出來：

`THRESHOLD=(8.0/24.0)*A+(40.0/168.0)*B+(80.0/336.0)*(1-A-B)`
- 8/24: 以大多上班或是活動時間計算，一天中的 8 小時
- 40/168: 一週工作時間，40 小時
- 80/336: 兩週工作時間，80 小時

In [ ]:
# Threshold Configuration
def SPATIAL_THRESHOLD(value):
    if value<12:
        return 6.6
    elif value<24:
        return 6.6
    elif value<36:
        return 9.35
    elif value<42:
        return 13.5
    elif value<48:
        return 17.0
    elif value<54:
        return 23.0
    elif value<59:
        return 27.5
    elif value<65:
        return 33.5
    elif value<71:
        return 40.5
    else:
        return 91.5
    

In [ ]:
# 帶入 Threshold
dfMean['PM_thr'] = dfMean['PM2.5'].apply(SPATIAL_THRESHOLD)

# 計算與目標日期的天數差
# 目標日期為 2022-10-29，需要將時間設定到 23:59:59 才不會出現小於一天的時間差
TARGET_DATE = "2022-10-29"
dfMean = dfMean.assign(days = lambda x: ( (pd.to_datetime(TARGET_DATE + " 23:59:59") - x.index.get_level_values('datetime')).days ) )

In [ ]:
dfMean

PM2.5        avg  neighbor_num  PM_thr  days
device_id    datetime                                                         
74DA38F207DE 2022-10-15 00:00:00   38.0  13.400000            10    13.5    14
             2022-10-15 00:05:00   37.0  19.888889             9    13.5    14
             2022-10-15 00:10:00   37.0  16.500000            12    13.5    14
             2022-10-15 00:20:00   36.0  16.750000             8    13.5    14
             2022-10-15 00:25:00   36.0  17.000000            11    13.5    14
...                                 ...        ...           ...     ...   ...
74DA38F210FE 2022-10-28 22:55:00    3.0  16.666667            12     6.6     1
             2022-10-28 23:00:00    3.0  19.642857            14     6.6     1
             2022-10-28 23:10:00    3.0  18.307692            13     6.6     1
             2022-10-28 23:20:00    2.0  19.545455            11     6.6     1
             2022-10-28 23:45:00    2.0  16.615385            13     6.6     1

[52523 rows x 5 columns]

In [ ]:
# 忽略鄰居數過少的資料
# 此範例目前設定為最少需要 2 個以上的鄰居來判斷
MINIMUM_NEIGHBORS = 2

# 當周圍平均數值高於自身一定程度，且鄰居數高於設定 -> indoor = True
dfMean["indoor"] = ((dfMean['avg'] - dfMean['PM2.5']) > dfMean['PM_thr']) & (dfMean['neighbor_num'] >= MINIMUM_NEIGHBORS)

# 當周圍平均數值低於自身一定程度，且鄰居數高於設定 -> emission = True
dfMean["emission"] = ((dfMean['PM2.5'] - dfMean['avg']) > dfMean['PM_thr']) & (dfMean['neighbor_num'] >= MINIMUM_NEIGHBORS)

In [ ]:
dfMean

PM2.5        avg  neighbor_num  PM_thr  \
device_id    datetime                                                      
74DA38F207DE 2022-10-15 00:00:00   38.0  13.400000            10    13.5   
             2022-10-15 00:05:00   37.0  19.888889             9    13.5   
             2022-10-15 00:10:00   37.0  16.500000            12    13.5   
             2022-10-15 00:20:00   36.0  16.750000             8    13.5   
             2022-10-15 00:25:00   36.0  17.000000            11    13.5   
...                                 ...        ...           ...     ...   
74DA38F210FE 2022-10-28 22:55:00    3.0  16.666667            12     6.6   
             2022-10-28 23:00:00    3.0  19.642857            14     6.6   
             2022-10-28 23:10:00    3.0  18.307692            13     6.6   
             2022-10-28 23:20:00    2.0  19.545455            11     6.6   
             2022-10-28 23:45:00    2.0  16.615385            13     6.6   

                                  days  indoor  emission  
device_id    datetime                                     
74DA38F207DE 2022-10-15 00:00:00    14   False      True  
             2022-10-15 00:05:00    14   False      True  
             2022-10-15 00:10:00    14   False      True  
             2022-10-15 00:20:00    14   False      True  
             2022-10-15 00:25:00    14   False      True  
...                                ...     ...       ...  
74DA38F210FE 2022-10-28 22:55:00     1    True     False  
             2022-10-28 23:00:00     1    True     False  
             2022-10-28 23:10:00     1    True     False  
             2022-10-28 23:20:00     1    True     False  
             2022-10-28 23:45:00     1    True     False  

[52523 rows x 7 columns]

In [ ]:
# 計算 2022.10.29
# DAY 01: 2022.10.28
# DAY 07: 2022.10.28 - 22
# DAY 14: 2022.10.28 - 15

# 初始化
dictIndoor = {iD: [] for iD in listId} # [1 day, 7 days, 14 days]
dictEmission = {iD: [] for iD in listId} # [1 day, 7 days, 14 days]

In [ ]:
# 將數值四捨五入到小數點後三位即可 (可做可不做)
# 如果被標記為 True 的區段沒有超過總區段的 1/3 則設定為 0，反之則紀錄原比例
basic = round(1.0/3.0, 3)

for iD in listId:
    dfId = dfMean.loc[iD]
    for day in [1, 7, 14]:
        indoor = (dfId[ dfId['days'] <= day]['indoor'].sum() / len(dfId[ dfId['days'] <= day])).round(3)
        dictIndoor[iD].append( 
            indoor if indoor > basic else 0
        )
        
        emission = (dfId[ dfId['days'] <= day]['emission'].sum() / len(dfId[ dfId['days'] <= day])).round(3)
        dictEmission[iD].append(
            emission if emission > basic else 0
        )
        
        

In [ ]:
dictIndoor

{'74DA38F207DE': [0, 0, 0],
 '74DA38F20A10': [0, 0, 0],
 '74DA38F20B20': [0.995, 0.86, 0.816],
 '74DA38F20B80': [0.995, 0.872, 0.867],
 '74DA38F20BB6': [0, 0, 0],
 '74DA38F20C16': [0.989, 0.535, 0],
 '74DA38F20D7C': [0, 0, 0],
 '74DA38F20D8A': [0, 0, 0],
 '74DA38F20DCE': [0, 0, 0],
 '74DA38F20DD0': [0.984, 0.871, 0.865],
 '74DA38F20DD8': [0, 0.368, 0.369],
 '74DA38F20DDC': [0, 0, 0],
 '74DA38F20DE0': [0, 0, 0],
 '74DA38F20DE2': [0, 0, 0],
 '74DA38F20E0E': [0, 0, 0],
 '74DA38F20E42': [0.99, 0.866, 0.87],
 '74DA38F20E44': [0, 0, 0],
 '74DA38F20F0C': [0.979, 0.872, 0.876],
 '74DA38F20F2C': [0, 0, 0],
 '74DA38F210FE': [0.99, 0.847, 0.864]}

In [ ]:
dictEmission

{'74DA38F207DE': [0.92, 0.737, 0.735],
 '74DA38F20A10': [0, 0, 0],
 '74DA38F20B20': [0, 0, 0],
 '74DA38F20B80': [0, 0, 0],
 '74DA38F20BB6': [0.492, 0.339, 0],
 '74DA38F20C16': [0, 0, 0],
 '74DA38F20D7C': [0.553, 0.342, 0.337],
 '74DA38F20D8A': [0.672, 0.457, 0.388],
 '74DA38F20DCE': [0.786, 0.556, 0.516],
 '74DA38F20DD0': [0, 0, 0],
 '74DA38F20DD8': [0, 0, 0],
 '74DA38F20DDC': [0.345, 0, 0],
 '74DA38F20DE0': [0.601, 0.503, 0.492],
 '74DA38F20DE2': [0, 0, 0],
 '74DA38F20E0E': [0.938, 0.75, 0.75],
 '74DA38F20E42': [0, 0, 0],
 '74DA38F20E44': [0.938, 0.69, 0.6],
 '74DA38F20F0C': [0, 0, 0],
 '74DA38F20F2C': [0.744, 0.575, 0.544],
 '74DA38F210FE': [0, 0, 0]}

In [ ]:
# 最終判斷

A=0.2
B=0.3
THRESHOLD=(8.0/24.0)*A+(40.0/168.0)*B+(80.0/336.0)*(1-A-B)

In [ ]:
# indoor

listIndoorDevice = []
for iD in listId:
    rate = A*dictIndoor[iD][0] + B*dictIndoor[iD][1] + (1-A-B)*dictIndoor[iD][2]
    if rate > THRESHOLD:
        listIndoorDevice.append( (iD, rate) )

In [ ]:
# emission

listEmissionDevice = []
for iD in listId:
    rate = A*dictEmission[iD][0] + B*dictEmission[iD][1] + (1-A-B)*dictEmission[iD][2]
    if rate > THRESHOLD:
        listEmissionDevice.append( (iD, rate) )

In [ ]:
listIndoorDevice

[('74DA38F20B20', 0.865),
 ('74DA38F20B80', 0.8941),
 ('74DA38F20C16', 0.3583),
 ('74DA38F20DD0', 0.8906),
 ('74DA38F20DD8', 0.2949),
 ('74DA38F20E42', 0.8928),
 ('74DA38F20F0C', 0.8954),
 ('74DA38F210FE', 0.8841)]

In [ ]:
listEmissionDevice

[('74DA38F207DE', 0.7726),
 ('74DA38F20D7C', 0.38170000000000004),
 ('74DA38F20D8A', 0.4655),
 ('74DA38F20DCE', 0.5820000000000001),
 ('74DA38F20DE0', 0.5171),
 ('74DA38F20E0E', 0.7876),
 ('74DA38F20E44', 0.6945999999999999),
 ('74DA38F20F2C', 0.5933)]